In [44]:
import numpy as np
import random
import platform
import sys
import os
import math
import time
import matplotlib
from matplotlib import pyplot as plt
import vtk
import pymp

In [27]:
raw_data_file = 'data/yA31_v02_300x300x300_99.vtk'
sampled_file = 'data/asteroid_sampled_hist_grad0.02.vtp'
var_name = 'vo2'
var_name1 = 'vo2'
query_th1 = 0.6
query_th2 = 1

In [28]:
## load data
reader = vtk.vtkGenericDataObjectReader()
reader.SetFileName(raw_data_file)
reader.Update()
raw_data = reader.GetOutput()

reader1 = vtk.vtkXMLPolyDataReader()
reader1.SetFileName(sampled_file)
reader1.Update()
sampled_pts = reader1.GetOutput()

In [21]:
def compute_3d_to_1d_map(x,y,z,dimx,dimy,dimz):
    index = x + dimx*(y+dimy*z)
    return index

In [22]:
extent = raw_data.GetExtent()
spacing = raw_data.GetSpacing()
origin = raw_data.GetOrigin()
dims = raw_data.GetDimensions()

query_pts = vtk.vtkPoints()
query_pts_physical = vtk.vtkPoints()
query_arr = vtk.vtkDoubleArray()
query_arr.SetName(var_name)

for i in range(sampled_pts.GetNumberOfPoints()):
    vo2 = sampled_pts.GetPointData().GetArray(var_name).GetTuple1(i)
    if vo2 >= query_th1 and vo2 <= query_th2:
        pts = sampled_pts.GetPoint(i)        
        ii = int((pts[0] - origin[0])/spacing[0] + 0.5)
        jj = int((pts[1] - origin[1])/spacing[1] + 0.5)
        kk = int((pts[2] - origin[2])/spacing[2] + 0.5)
        query_pts.InsertNextPoint([ii,jj,kk])
        query_arr.InsertNextTuple1(vo2)   
        query_pts_physical.InsertNextPoint(pts)
        
print (query_pts.GetNumberOfPoints())  

pdata = vtk.vtkPolyData()
pdata.SetPoints(query_pts_physical)
pdata.GetPointData().AddArray(query_arr)

writer1 = vtk.vtkXMLPolyDataWriter()
writer1.SetInputData(pdata)
writer1.SetFileName('output/recon_query.vtp')
writer1.Write()  

216058


1

In [50]:
new_data = vtk.vtkImageData()
new_data.SetDimensions(dims)
new_data.SetSpacing(spacing)
new_data.SetOrigin(origin)

query_arr_new = vtk.vtkDoubleArray()
query_arr_new.SetName(var_name)
query_arr_new.SetNumberOfTuples(dims[0]*dims[1]*dims[2])

query_arr_new1 = vtk.vtkDoubleArray()
query_arr_new1.SetName(var_name1)
query_arr_new1.SetNumberOfTuples(dims[0]*dims[1]*dims[2])

            
for p in range(query_pts.GetNumberOfPoints()):   
    pts = query_pts.GetPoint(p)
    val = query_arr.GetTuple1(p)
    index = compute_3d_to_1d_map(int(pts[0]),int(pts[1]),int(pts[2]),dims[0],dims[1],dims[2])
    query_arr_new.SetTuple1(index,val)
    
new_data.GetPointData().AddArray(query_arr_new) 

0

In [31]:
for k in range(1,dims[2]-1):
    for j in range(1,dims[1]-1):
        for i in range(1,dims[0]-1):
            
            index = compute_3d_to_1d_map(i,j,k,dims[0],dims[1],dims[2])
            val = query_arr_new.GetTuple1(index)
            
            sum_val = 0.0
            if val == 0.0:
                index1 = compute_3d_to_1d_map(i-1,j,k,dims[0],dims[1],dims[2])
                
                #print i-1,j,k,dims[0],dims[1],dims[2] 
                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                
                index1 = compute_3d_to_1d_map(i+1,j,k,dims[0],dims[1],dims[2])
                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                
                index1 = compute_3d_to_1d_map(i,j-1,k,dims[0],dims[1],dims[2])
                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                
                index1 = compute_3d_to_1d_map(i,j+1,k,dims[0],dims[1],dims[2])
                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                
                index1 = compute_3d_to_1d_map(i,j,k-1,dims[0],dims[1],dims[2])
                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                
                index1 = compute_3d_to_1d_map(i,j,k+1,dims[0],dims[1],dims[2])
                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                
                sum_val = sum_val / 8.0
                
                query_arr_new1.SetTuple1(index,sum_val)
                                
            else:
                query_arr_new1.SetTuple1(index,val)

new_data.GetPointData().AddArray(query_arr_new1)                

0

In [52]:
window=2
for k in range(window,dims[2]-window):
    for j in range(window,dims[1]-window):
        for i in range(window,dims[0]-window):
            
            index = compute_3d_to_1d_map(i,j,k,dims[0],dims[1],dims[2])
            val = query_arr_new.GetTuple1(index)
            
            sum_val = 0.0
            if val == 0.0:
                
                count=0
                for kk in range(k-window,k+window,1):
                    for jj in range(j-window,j+window,1):
                        for ii in range(i-window,i+window,1):
                            
                            if kk == k and ii== i and jj == j: ## ignore the central point
                                continue
                            else:    

                                index1 = compute_3d_to_1d_map(ii,jj,kk,dims[0],dims[1],dims[2])
                                sum_val = sum_val + query_arr_new.GetTuple1(index1)
                                count=count+1
                            
                sum_val = sum_val / count
                
                query_arr_new1.SetTuple1(index,sum_val)
                        
            else:
                query_arr_new1.SetTuple1(index,val)

new_data.GetPointData().AddArray(query_arr_new1)                        

0

In [51]:
writer = vtk.vtkXMLImageDataWriter()
writer.SetInputData(new_data)
writer.SetFileName('output/recon_query1.vti')
writer.Write()           

1